# SC2 replay stats server

Scans the players replay folder and compiles win/loss stats for the current season.

This notebook is for experimenting with the result table. 

In [3]:
import sc2reader
import pandas as pd
import glob
import os
import datetime

In [4]:
REPLAY_FOLDER = r'.'

PLAYER = 'XXX'

RACES = {
    'Zerg': 'Z',
    'Terran': 'T',
    'Protoss': 'P'
}

Parse all replay files. This takes about .5 seconds per replay, be patient

In [13]:
list_of_files = glob.glob(os.path.join(REPLAY_FOLDER, '*.SC2Replay'))

#today = datetime.date.today()
today = datetime.date(today.year, 1, 7)

list_of_files = [f for f in list_of_files if today <= datetime.date.fromtimestamp(os.path.getmtime(f))]

data = []
for file_path in list_of_files:
    replay = sc2reader.load_replay(file_path)
    if replay.category == 'Ladder' and replay.game_type == '1v1': 
        row = {
            'Map': replay.map_name, 
            'Player1': replay.players[0].name, 
            'Player1_Race': RACES[replay.players[0].play_race], 
            'Player1_Result': replay.players[0].result == 'Win', 
            'Player2': replay.players[1].name, 
            'Player2_Race': RACES[replay.players[1].play_race],
            'Player2_Result': replay.players[1].result == 'Win' 
        }
        data.append(row)
df = pd.DataFrame(data, columns=['Map', 'Player1', 'Player1_Race', 'Player1_Result', 'Player2', 'Player2_Race', 'Player2_Result'])
df

,Map,Player1,Player1_Race,Player1_Result,Player2,Player2_Race,Player2_Result
0,2000 Atmospheres LE,polymath,T,True,zatic,Z,False
1,2000 Atmospheres LE,SEINAMENT,Z,False,zatic,P,True
2,2000 Atmospheres LE,ADG,P,False,zatic,Z,True
3,2000 Atmospheres LE,PapstBuk,P,False,zatic,Z,True
4,2000 Atmospheres LE,UncleKev,Z,True,zatic,Z,False
...,...,...,...,...,...,...,...
108,Pride of Altaris LE,zatic,Z,True,Svagu,T,False
109,Pride of Altaris LE,zatic,Z,False,Shekowaffle,T,True
110,Pride of Altaris LE,zatic,Z,True,SkyTER,T,False
111,Pride of Altaris LE,zatic,Z,False,Wormms,T,True


Filter out all ZvX replays of PLAYER, and display them in a table. 

Need to run add PvZ to ZvP to have a neatly formatted table. 

In [14]:
player_as_zerg1 = (df['Player1'] == PLAYER) & (df['Player1_Race'] == 'Z')

In [15]:
player_as_zerg2 = (df['Player2'] == PLAYER) & (df['Player2_Race'] == 'Z')

In [16]:
p1 = df[player_as_zerg1].groupby(by=['Player1_Race', 'Player2_Race']).sum()
p1

Player1_Result  Player2_Result
Player1_Race Player2_Race                                
Z            P                          4               4
             T                         17              12
             Z                         11               6

In [17]:
p2 = df[player_as_zerg2].groupby(by=['Player1_Race', 'Player2_Race']).sum()
p2

,,Player1_Result,Player2_Result
Player1_Race,Player2_Race,,
P,Z,4,6
T,Z,7,10
Z,Z,6,6


In [18]:
p3 = p2.rename(columns={'Player2_Result':'Player1_Result', 'Player1_Result':'Player2_Result'})
p3

,,Player2_Result,Player1_Result
Player1_Race,Player2_Race,,
P,Z,4,6
T,Z,7,10
Z,Z,6,6


In [19]:
p = p1.copy()

if ('Z', 'P') not in p.index:
    p.loc[('Z', 'P'),:] = 0
if ('Z', 'T') not in p.index:
    p.loc[('Z', 'T'),:] = 0
if ('Z', 'Z') not in p.index:    
    p.loc[('Z', 'Z'),:] = 0

if ('P','Z') not in p3.index:
    p3.loc[('P','Z'),:] = 0
if ('T','Z') not in p3.index:
    p3.loc[('T','Z'),:] = 0
if ('Z', 'Z') not in p3.index:    
    p3.loc[('Z', 'Z'),:] = 0


In [20]:

p.loc[('Z', 'P')] += p3.loc[('P','Z')]
p.loc[('Z', 'T')] += p3.loc[('T','Z')]
p.loc[('Z', 'Z')] += p3.loc[('Z','Z')]
p

Player1_Result  Player2_Result
Player1_Race Player2_Race                                
Z            P                         10               8
             T                         27              19
             Z                         17              12